In [1]:
pip install transformers datasets torch

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00


In [2]:
from transformers import BertTokenizerFast, BertForQuestionAnswering, AdamW
from datasets import load_dataset
import torch

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

squad = load_dataset("squad")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [3]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples['question']]
    inputs = tokenizer(
        questions,
        examples['context'],
        max_length=384,
        truncation=True,
        return_offsets_mapping=True,
        padding="max_length",
        return_tensors="pt"
    )
    start_positions = []
    end_positions = []

    for i, offset_mapping in enumerate(inputs["offset_mapping"]):
        answer = examples["answers"][i]["text"][0]
        start_char = examples["answers"][i]["answer_start"][0]
        end_char = start_char + len(answer)
        sequence_ids = inputs.sequence_ids(i)
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        start_position = context_start
        end_position = context_end

        for idx, mapping in enumerate(offset_mapping):
            if mapping[0] <= start_char < mapping[1]:
                start_position = idx
            if mapping[0] < end_char <= mapping[1]:
                end_position = idx
                break

        start_positions.append(start_position)
        end_positions.append(end_position)

    inputs["start_positions"] = torch.tensor(start_positions)
    inputs["end_positions"] = torch.tensor(end_positions)
    return inputs
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [4]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataset = tokenized_squad["train"].select(range(1000))
val_dataset = tokenized_squad["validation"].select(range(100))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=data_collator)


In [5]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
for epoch in range(3):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items() if k != "offset_mapping"}

        if "start_positions" not in inputs or "end_positions" not in inputs:
            continue

        outputs = model(**inputs)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Loss: 6.129158973693848
Loss: 5.986989498138428
Loss: 5.827647686004639
Loss: 5.770813941955566
Loss: 5.706445693969727
Loss: 5.673677444458008
Loss: 5.447864532470703
Loss: 5.624608993530273
Loss: 5.397456169128418
Loss: 5.426306247711182
Loss: 4.7589616775512695
Loss: 5.12163782119751
Loss: 5.132762908935547
Loss: 4.956883430480957
Loss: 4.729907035827637
Loss: 4.627170562744141
Loss: 5.021705627441406
Loss: 4.4247589111328125
Loss: 5.043135643005371
Loss: 4.644132614135742
Loss: 4.391881465911865
Loss: 4.382035255432129
Loss: 4.337233543395996
Loss: 3.9323668479919434
Loss: 4.056867599487305
Loss: 4.02902889251709
Loss: 4.0959577560424805
Loss: 4.386721611022949
Loss: 4.266845703125
Loss: 4.4077301025390625
Loss: 4.31691837310791
Loss: 4.4334211349487305
Loss: 4.849624156951904
Loss: 4.388615131378174
Loss: 4.193485260009766
Loss: 4.311905384063721
Loss: 3.8721556663513184
Loss: 3.5456042289733887
Loss: 3.6210484504699707
Loss: 3.477184295654297
Loss: 4.2562055587768555
Loss: 4.3993

In [6]:
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors='pt', max_length=384, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits)

    answer = tokenizer.decode(inputs['input_ids'][0][answer_start:answer_end+1], skip_special_tokens=True)
    return answer



In [7]:
context = input("Enter passage:")
question = input("Enter question:")

print(answer_question(question, context))

Enter passage:Taylor Alison Swift (born December 13, 1989) is an American singer-songwriter. Known for her biographical songwriting, artistic reinventions, and cultural impact, Swift is a leading figure in popular music and the subject of widespread public interest.
Enter question:What taylor swift is known for ?
singer - songwriter. known for her biographical songwriting
